In [ ]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data

In [ ]:
dlm = importr('dlm')

R Code
```R
rw <- dlm(m0 = 0, C0 = 10, FF = 1, V = 1.4, GG = 1, W = 0.2)
```

In [ ]:
rw  = dlm.dlm(m0 = 0, C0 = 10, FF = 1, V = 1.4, GG = 1, W = 0.2)

In [ ]:
rw

In [ ]:
rw[0]

In [ ]:
rw.rx2('m0')

In [ ]:
rw[1]

In [ ]:
rw.rx2('C0')

In [ ]:
{k:v for k,v in rw.items()}['m0']

In [ ]:
{k:v for k,v in rw.items()}['C0']

In [ ]:
import numpy as np
np.matrix(rw.rx2('C0'))

In [ ]:
rw.rx2('W')

In [ ]:
x = robjects.ListVector({'a': 1, 'b': 2, 'c': 3})

In [ ]:
print(x)

In [ ]:
x.names.index('b')

In [ ]:
x[x.names.index('b')] = 9

In [ ]:
x

In [ ]:
print(x)

R code:

```R
lg <- dlm(m0 = rep(0,2), 
          C0 = 10 * diag(2), 
          FF = matrix(c(1,0),nr=1), 
          V = 1.4, GG = matrix(c(1,0,1,1),nr=2), 
          W = diag(c(0,0.2)))
```

In [ ]:
m0 = robjects.r.matrix(robjects.FloatVector([0,0]), nrow=2)
C0 =  robjects.r.matrix(robjects.FloatVector(list(10 * np.eye(2).ravel())), nrow=2)
FF =  robjects.r.matrix(robjects.FloatVector([1, 0]), nrow=1)
GG =  robjects.r.matrix(robjects.FloatVector([1, 0, 1, 1]), nrow=2)
W =  robjects.r.matrix(robjects.FloatVector(list(np.diag([0, 0.2]).ravel())), nrow=2)
print(W)
lg  = dlm.dlm(m0 = m0 , 
              C0 = C0, 
              FF = FF, 
              V = 1.4, 
              GG = GG, 
              W = W)

In [ ]:
print(lg)

In [ ]:
print(lg.names)
print(lg.names.index("W"))
print(lg[lg.names.index("W")])
print(lg.rx2('W'))

### Dynamic regression model

$$
\begin{align}
Y_t &= \theta_1 + \theta_2 x_t + \epsilon_t\\
\begin{bmatrix} 
    \theta_1 \\ \theta_2 \end
{bmatrix}_t &=  \begin{bmatrix}
                1 & 0\\
                0 & 1
               \end{bmatrix} \begin{bmatrix} 
                                 \theta_1 \\ \theta_2
                              \end{bmatrix}_{t-1}   +  \epsilon  
\end{align}
$$

```R
x <- rnorm(100) # covariates
dlr <- dlm(m0 = rep(0,2), 
           C0 = 10 * diag(2), 
           FF = matrix(c(1,0),nr=1),
           V = 1.3, 
           GG = diag(2), 
           W = diag(c(0.4,0.2)),
           JFF = matrix(c(0,1),nr=1), 
           X = as.matrix(x))
```

In [ ]:
# Time varying x
x = np.random.randn(100)
m0 = robjects.r.matrix(robjects.FloatVector([0,0]), nrow=2)
C0 =  robjects.r.matrix(robjects.FloatVector(list(10 * np.eye(2).ravel())), nrow=2)
FF =  robjects.r.matrix(robjects.FloatVector([1, 0]), nrow=1)
V = 1.3
GG =  robjects.r.matrix(robjects.FloatVector(np.eye(2).ravel()), nrow=2)
W =  robjects.r.matrix(robjects.FloatVector(np.diag([0.4, 0.2]).ravel()), nrow = 2)
JFF =  robjects.r.matrix(robjects.FloatVector([0, 1]), nrow = 1)
X =  robjects.r.matrix(robjects.FloatVector(x), ncol = 1)


dlr = dlm.dlm(m0 = m0, 
           C0 = C0, 
           FF = FF,
           V = V, 
           GG = GG, 
           W =W,
           JFF = JFF, 
           X = X)


In [ ]:
print(dlr)

In [ ]:
# Note we put second elemnt of FF 0 as it is time varying and is set in X
# We can see JFF.. match with $X above.. 
x[0:10]

### R Code

```R
#-------------------------------------------------------------------------------
#Generate some data first - toy example
#Kalman filter/dynamic linear model in R
#-------------------------------------------------------------------------------
rm(list=ls())
set.seed(100)
df <- data.frame(x1 = sample(seq(from = -1.0, by = 0.01, length.out = 1000), 
                               size = 1000, replace = FALSE),
                 x2 = sample(seq(from = -3.0, by = 0.01, length.out = 1000), 
                               size = 1000, replace = FALSE))

theta1 <- 0.4
theta2 <- 1.2
y      <- sum(df[1, ] * c(theta1[1], theta2[1])) + rnorm(1, mean = 0, sd = 0.01)

for(i in 2:1000)
{
  theta1 <- c(theta1, theta1[i-1] + rnorm(1, mean = 0, sd = 0.01))
  theta2 <- c(theta2, theta2[i-1] + rnorm(1, mean = 0, sd = 0.02))
  y      <- c(y, sum(df[i, ] * c(theta1[i], theta2[i])) + rnorm(1, mean = 0, sd = 0.01))
}


df[["y"]] <- y
df[["tim"]] <- 1:length(y)
```

In [ ]:
import pandas as pd
np.random.seed(144)

#SIZE = 1000

SIZE = 1000


# Not exactly above.. but
df = pd.DataFrame({
              "x1": np.random.choice(np.linspace(-1, 1, num=SIZE), size=SIZE, replace=False),
              "x2": np.random.choice(np.linspace(-3, 3, num=SIZE), size=SIZE, replace=False)
             })

theta1 = 0.4
#theta2 = 1.2

theta2 = -0.5

# Theta evolution
theta_1_shocks = np.random.normal(loc=0.0,
                                  scale=0.01,
                                  size=df.shape[0]-1)

theta_2_shocks = np.random.normal(loc=0.0,
                                  scale=0.02,
                                  size=df.shape[0]-1)

theta_1_shocks = np.insert(theta_1_shocks, 0,0)
theta1_s = theta1 + theta_1_shocks.cumsum()
theta_2_shocks = np.insert(theta_2_shocks, 0,0)
theta2_s = theta2 + theta_2_shocks.cumsum()

y_shocks = np.random.normal(loc=0.0, scale=0.01, size=df.shape[0])

y = np.apply_along_axis(sum, 1, df[["x1", "x2"]] * np.vstack([theta1_s, theta2_s]).T) + y_shocks

df["y"] = y
df["theta1"] = theta1_s
df["theta2"] = theta2_s

In [ ]:
df

##### R Code

```R
#-------------------------------------------------------------------------------
#Learn MLE from first 500
#-------------------------------------------------------------------------------

buildFn <- function(x)
{
  ret.mod <- dlm::dlmModReg(X = as.matrix(df[1:500, c("x1", "x2")]),
                            dV = exp(x[3]), 
                            dW = exp(x[1:2]), 
                            addInt = FALSE)
  ret.mod
}

fit <- dlm::dlmMLE( y     = df$y[1:500], 
                    build = buildFn, 
                    parm  = log(c(rep(1e2, 2), 1e2)),
                    lower = log(rep(1e-5, 3)), 
                    hessian=TRUE)


mod.res <- buildFn(fit$par)
avarLog <- solve(fit$hessian)
avar <- diag(exp(fit$par)) %*% avarLog %*% diag(exp(fit$par)) # Delta method
V(mod.res)
W(mod.res)

```


# https://rpy2.github.io/doc/v3.5.x/html/robjects_functions.html

Seems much of work to integrate R's dlm from python... let me stop here and look for proper implementation of library
found one here:
https://brandonwillard.github.io/dynamic-linear-models-in-theano.html

-- But let me start working on my old idea :)

In [ ]:
r_func_code = """

function(x){
   c(mean(x), sd(x))
}
"""

r_func = robjects.r(r_func_code)
py_func = robjects.functions.wrap_r_function(r_func, 'py_func')

In [ ]:
py_func(robjects.FloatVector([1,2,3]))

In [ ]:
# Try decorator/closure kind - 
r_func_code = """
function(const){
  finternal <- function(x){
    c(mean(x) + const, sd(x) + const)
  }
  finternal
}
"""

r_func = robjects.r(r_func_code)
py_func_g = robjects.functions.wrap_r_function(r_func, 'g')


In [ ]:
# Direct Call
print(py_func_g(10)(robjects.FloatVector([1,2,3])))

# Put differently
temp_f = py_func_g(10)
print(temp_f(robjects.FloatVector([1,2,3])))

## I try my R code..

https://github.com/jaivrat/R-codes/blob/ea1ad3c256fb0e64285d1e5e7ce8aa3852c64262/dlm-dynamic-linear-model.R


In [ ]:
## We try MLE estimator of Variance matrices
# I think we can do exponential moving average estimator of covariance matrix as estimate of V then 
# we may not need MLE estimator

r_func_code_mle = """
function(df){
  # MLE estimator
  buildFn <- function(x)
  {
      ret.mod <- dlm::dlmModReg(X = as.matrix(df[c("x1", "x2")]),
                            dV = exp(x[3]), 
                            dW = exp(x[1:2]), 
                            addInt = FALSE)
      ret.mod
  }
  
  fit <- dlm::dlmMLE( y  = df$y, 
                    build = buildFn, 
                    parm  = log(c(rep(1e2, 2), 1e2)),
                    lower = log(rep(1e-5, 3)), 
                    hessian=TRUE)
  mod.res <- buildFn(fit$par)
  avarLog <- solve(fit$hessian)
  avar <- diag(exp(fit$par)) %*% avarLog %*% diag(exp(fit$par))  
  return(list(mod.res = mod.res, V = V(mod.res), W =  W(mod.res)))
}
"""

r_func_mle = robjects.r(r_func_code_mle)

py_func_mle = robjects.functions.wrap_r_function(r_func_mle, 'mle')


In [ ]:
# df

In [ ]:
d = {}
for col in df.columns:
    print(col)
    d[col] = robjects.FloatVector(df[col].values)

df_r = robjects.DataFrame(d)

In [ ]:
print(df_r)

In [ ]:
#SIZE

In [ ]:
subset_for_mle = df_r.rx(robjects.IntVector(range(0,1 + int(SIZE/2))), True)
print(subset_for_mle)

In [ ]:
df.to_clipboard()

In [ ]:
## dict(df)
result_mle = py_func_mle(subset_for_mle)

In [ ]:
print(result_mle.names)
print("V:")
print(result_mle.rx2('V'))
print("W:")
print(result_mle.rx2('W'))
print("mod.res:")
print(result_mle.rx2('mod.res'))

In [ ]:

r_func_code_filter = """
function(df, mod.res){
  res.filt <- dlm::dlmFilter(y = df$y, mod = mod.res)
  results.df <- data.frame(res.filt$m[-1, ] , 
                         theta1 = df$theta1, 
                         theta2 = df$theta2
                         )
  return(list(results.df =  results.df, res.filt = res.filt))
}
"""

r_func_filter = robjects.r(r_func_code_filter)
py_func_filter = robjects.functions.wrap_r_function(r_func_filter, 'filter')


In [ ]:
res_df_robj = py_func_filter(subset_for_mle, result_mle.rx2('mod.res'))
res_filt    = res_df_robj.rx2("res.filt")
results_df  = res_df_robj.rx2("results.df")

# Cover to python data frame
res_df =  pd.DataFrame(results_df).T
res_df =  res_df.set_axis([x for x in results_df.names], axis = 1)

# Actually first 1 values are filtered state vectors res.filt$m, ie filtered theta1 and theta2
res_df

X1, X2 (not confuse with x1 and x2 inputs) are filtered values of internal states.. ie thet1 and theta2

In [ ]:
## We can try seeing filtered values and compare with actual thetas
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
fig = res_df.plot()
fig.show()


```R
#-------------------------------------------------------------------------------
#Future prediction though kalman filter
#-------------------------------------------------------------------------------
ret.mod.fwd <- dlm::dlmModReg(X = as.matrix(df[-(1:500), c("x1", "x2")]),
                              dV = V(mod.res), 
                              dW = diag(W(mod.res)), 
                              m0 = as.numeric(tail(res.filt$m, 1)),
                              C0 = with(res.filt, dlmSvd2var(U.C[[501]], D.C[501,])),
                              addInt = FALSE)

res.fwd.filt   <- dlm::dlmFilter(y = df$y[-(1:500)], mod = ret.mod.fwd)

results.fwd.df <- data.frame(res.fwd.filt$m[-1, ] , 
                             theta1 = theta1[-(1:500)], 
                             theta2 = theta2[-(1:500)],
                             tim = df$tim[-(1:500)])

results.fwd.df.melt <- reshape2::melt(results.fwd.df, id.vars = "tim")     
ggplot(data = results.fwd.df.melt, aes(x = tim, y = value, group = variable, colour = variable)) + 
  geom_line() + 
  ggtitle("States, ie thetas")
```


In [ ]:
df.shape[0]

In [ ]:

r_func_code_pred_filter = """
function(df, mod.res, res.filt){
  ret.mod.fwd <- dlm::dlmModReg(X = as.matrix(df[c("x1", "x2")]),
                              dV = V(mod.res), 
                              dW = diag(W(mod.res)), 
                              m0 = as.numeric(tail(res.filt$m, 1)),
                              C0 = with(res.filt, dlmSvd2var(U.C[[nrow(df)/2 - 1]], D.C[nrow(df)/2,])),
                              addInt = FALSE)
  res.fwd.filt   <- dlm::dlmFilter(y = df$y, mod = ret.mod.fwd)
  results.fwd.df <- data.frame(res.fwd.filt$m[-1, ] ,
                             theta1 = df$theta1, 
                             theta2 = df$theta2)

  return(list(results.fwd.df =  results.fwd.df, res.fwd.filt = res.fwd.filt))
  
  
}
"""
# Have a look https://rdrr.io/cran/dlm/man/dlmFilter.html

r_func_pred_filter = robjects.r(r_func_code_pred_filter)
py_func_pred_filter = robjects.functions.wrap_r_function(r_func_pred_filter, 'pred_filter')

In [ ]:
print(res_filt.names)
res_filt.rx2("U.C")[5]
res_filt.rx2("D.C")

In [ ]:
results_pred_filter = py_func_pred_filter(df_r, result_mle.rx2('mod.res'), res_filt )

res_fwd_filt = results_pred_filter.rx2('res.fwd.filt')
results_fwd = results_pred_filter.rx2('results.fwd.df')

# Covert to python data frame
results_fwd_df = pd.DataFrame(results_fwd).T
results_fwd_df = results_fwd_df.set_axis([x for x in results_fwd.names], axis = 1)
results_fwd_df

In [ ]:
results_fwd_df.to_clipboard()

In [ ]:
# X1 and X2 are filtered theta1 and theta2
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
fig = results_fwd_df.plot()
fig.show()

In [ ]:
# Actually y is not important here as x_t's vary and prediction is not relevant! We are more after state estimates!
forecasted_y = np.array(res_fwd_filt.rx2("f"))

In [ ]:
# X1 and X2 are filtered theta1 and thta2
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
fig = pd.DataFrame({"y":df["y"], 
                    "forecasted_y": pd.Series(forecasted_y).shift(-1).values }).plot()
fig.show()

Actually y is not important here as x_t's vary and prediction is not relevant! We are more after state estimates!

In [ ]:
# X1 and X2 are filtered theta1 and thta2
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
fig = results_fwd_df[["X1", "theta1"]].plot()
fig.show()

In [ ]:
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"
fig = results_fwd_df[["X2", "theta2"]].plot()
fig.show()